In [ ]:
!pip install transformers
!pip install -q transformers datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.6 MB/s eta 0:00:00


In [ ]:
import pandas as pd

train_df = pd.read_csv('non_skewed_train.csv')
test_df = pd.read_csv('non_skewed_test.csv')
df = pd.concat([train_df, test_df])
df

,id,text,label
0,86426.0,<user> ask native americans take,1
1,90194.0,<user> <user> go home drunk <user> maga trump2...,1
2,16820.0,amazon investigating chinese employees selling...,0
3,62688.0,<user> someone vetaken piece shit volcano <emo...,1
4,97670.0,<user> liberals kookoo,1
...,...,...,...
3531,NaN,eagles fucking trash,1
3532,78174.0,<user> <user> fraud amp fight anti semitism ex...,1
3533,95459.0,<user> thank jourdan amp hayley <emoji : face_...,0
3534,64878.0,<user> <user> fake news term used rather word ...,0


In [ ]:
df = df.sample(frac=1).reset_index(drop=True)
df

,id,text,label
0,21478.0,<user> bitches brain,1
1,36130.0,<user> <user> people hold hold 1st photo beati...,0
2,69456.0,<user> <user> maybe wrong take oath preserve u...,1
3,65130.0,<user> possible california implemented strict ...,1
4,84486.0,<user> inviting war criminals canada parliamen...,0
...,...,...,...
17675,84558.0,<user> <user> video worth,0
17676,74733.0,<user> <user> way democrats smeared conservati...,1
17677,50361.0,<user> war conservatism antifa,0
17678,85283.0,<user> antifa domestic terror organisation though,0


In [ ]:
test_df = df.sample(frac = 0.2, random_state = 42)
train_df = df.drop(test_df.index)

In [ ]:
train_df

,id,text,label
1,36130.0,<user> <user> people hold hold 1st photo beati...,0
2,69456.0,<user> <user> maybe wrong take oath preserve u...,1
4,84486.0,<user> inviting war criminals canada parliamen...,0
6,81472.0,<user> never pants forever throwing people eye...,1
7,77694.0,<user> <user> <user> course swalwell pencil ne...,0
...,...,...,...
17674,NaN,8220 <user> hoes messy god 8221 nbs 128064,1
17675,84558.0,<user> <user> video worth,0
17676,74733.0,<user> <user> way democrats smeared conservati...,1
17677,50361.0,<user> war conservatism antifa,0


In [ ]:
train_df.to_csv('non_skewed_train.csv', index = False)
test_df.to_csv('non_skewed_test.csv', index = False)

In [ ]:
from datasets import load_dataset
dataset = load_dataset('csv', data_files={'train': 'non_skewed_train.csv', 'test': 'non_skewed_test.csv'})

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: [' id', 'text', 'label'],
        num_rows: 14144
    })
    test: Dataset({
        features: [' id', 'text', 'label'],
        num_rows: 3536
    })
})

In [ ]:
dataset = dataset.remove_columns([' id'])

In [ ]:
dataset['train'][0]

{'text': '<user> <user> people hold hold 1st photo beating republican korean war vet 2nd amp 3rd photo antifa beating minorities lgbt kids conservative left left shutting free speech url',
 'label': 0}

In [ ]:
dataset['train'].features.keys()

dict_keys(['text', 'label'])

In [ ]:
from transformers import AutoTokenizer
import numpy as np

tokenizer = AutoTokenizer.from_pretrained("FacebookAI/roberta-large")

def preprocess_data(examples):
  # take a batch of texts
  text = examples["text"]
  # encode them
  encoding = tokenizer(text, padding="max_length", truncation=True, max_length = 30)
  return encoding

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [ ]:
encoded_dataset = dataset.map(preprocess_data, batched=True)

Map:   0%|          | 0/14144 [00:00<?, ? examples/s]

Map:   0%|          | 0/3536 [00:00<?, ? examples/s]

In [ ]:
train_ds = encoded_dataset["train"].shuffle(seed=42)
val_ds = encoded_dataset["test"].shuffle(seed=42)

In [ ]:
train_ds.set_format("torch")
val_ds.set_format("torch")

In [ ]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("FacebookAI/roberta-large",
                                                           num_labels=2)

model.safetensors:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at FacebookAI/roberta-large and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
batch_size = 8
metric_name = "f1"

In [ ]:
!pip install transformers[torch]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 2.2 MB/s eta 0:00:00


In [ ]:
!pip install accelerate -U

In [ ]:
import accelerate

accelerate.__version__

'0.27.2'

In [ ]:
from transformers import TrainingArguments, Trainer

args = TrainingArguments(
    f"roberta-large-olid",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,  # Adjusted batch size
    per_device_eval_batch_size=8,  # Adjusted batch size
    num_train_epochs=5,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    gradient_accumulation_steps=2,  # Gradient accumulation
    logging_steps=100,  # Adjust as needed
    eval_steps=200,  # Adjust as needed
)

In [ ]:
!pip install evaluate -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 1.2 MB/s eta 0:00:00


In [ ]:
from transformers import AutoTokenizer, AdamW, get_scheduler
optimizer = AdamW(model.parameters(), lr=args.learning_rate, weight_decay=args.weight_decay)
scheduler = get_scheduler(
    "linear",
    optimizer,
    num_warmup_steps=0,
    num_training_steps=len(train_ds) // args.gradient_accumulation_steps * args.num_train_epochs,
)

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:429: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [ ]:
from sklearn.metrics import f1_score, roc_auc_score, accuracy_score
import evaluate

metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    if isinstance(logits, tuple):
      logits = logits[0]
    predictions = np.argmax(logits, axis=-1)
    acc = accuracy_score(predictions, labels)
    f1 = f1_score(predictions, labels, average = 'macro')

    return {"accuracy": acc, "f1": f1}

In [ ]:
train_ds

Dataset({
    features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 43516
})

In [ ]:
trainer = Trainer(
    model,
    args,
    train_dataset=train_ds,
    eval_dataset=val_ds,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    optimizers=(optimizer, scheduler),
)

In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.450100,0.390560,0.837387,0.837284
2,0.382600,0.394871,0.817308,0.815703
3,0.360400,0.380970,0.840498,0.840385
4,0.345600,0.402708,0.841912,0.841912
5,0.278400,0.494204,0.843891,0.843840


TrainOutput(global_step=4420, training_loss=0.36080129017117873, metrics={'train_runtime': 2329.4937, 'train_samples_per_second': 30.359, 'train_steps_per_second': 1.897, 'total_flos': 3861690586790400.0, 'train_loss': 0.36080129017117873, 'epoch': 5.0})

In [ ]:
preds = trainer.predict(val_ds)

In [ ]:
val_ds[0]

{'text': 'pugchat wanted let know may able see pugchat buddies tonight grandma working late plus going bakery order cake mom birthday weekend <emoji : slightly_smiling_face>',
 'label': tensor(0),
 'input_ids': tensor([    0,   642,  3252, 29465,   770,   905,   216,   189,   441,   192,
           181,  3252, 29465, 30489,  3422, 30894,   447,   628,  2704,   164,
         22428,   645,  8492,  3795,  4115,   983, 28696,   991, 16873,     2]),
 'attention_mask': tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1])}

In [ ]:
preds

PredictionOutput(predictions=array([[ 2.422389  , -1.762017  ],
       [-0.95131   , -0.04846925],
       [ 2.2868931 , -1.6009687 ],
       ...,
       [ 2.472857  , -1.8652289 ],
       [-1.2301286 ,  0.5511303 ],
       [-2.6125944 ,  1.7951086 ]], dtype=float32), label_ids=array([0, 0, 0, ..., 0, 0, 1]), metrics={'test_loss': 0.49420425295829773, 'test_accuracy': 0.8438914027149321, 'test_f1': 0.8438402457757297, 'test_runtime': 23.4777, 'test_samples_per_second': 150.611, 'test_steps_per_second': 18.826})

In [ ]:
pred_vals = np.argmax(preds.predictions, axis=-1)
labels = preds.label_ids
pred_vals, labels

(array([0, 1, 0, ..., 0, 1, 1]), array([0, 0, 0, ..., 0, 0, 1]))

In [ ]:
from sklearn.metrics import f1_score, roc_auc_score, accuracy_score
print('roc: ', roc_auc_score(labels, pred_vals))
print('f1: ', f1_score(labels, pred_vals, average = 'macro'))
print('accuracy: ', accuracy_score(labels, pred_vals))

roc:  0.8445549630844955
f1:  0.8438402457757297
accuracy:  0.8438914027149321


In [ ]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) y
Token is valid (permission: write).
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub.
Run the following command in your terminal in case you want to set the 'store' creden

In [ ]:
trainer.push_to_hub('roberta-large-olid')

training_args.bin:   0%|          | 0.00/4.92k [00:00<?, ?B/s]

Upload 3 LFS files:   0%|          | 0/3 [00:00<?, ?it/s]

events.out.tfevents.1709649172.647d8b11de21.1471.1:   0%|          | 0.00/16.1k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/ARC4N3/roberta-large-olid/commit/d51c3ea7f773140a4f26fe1c7e1c546d90ed048b', commit_message='roberta-large-olid', commit_description='', oid='d51c3ea7f773140a4f26fe1c7e1c546d90ed048b', pr_url=None, pr_revision=None, pr_num=None)